https://github.com/philipperemy/keras-tcn 

In [3]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)

###########################################################
# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("CUDA_VISIBLE_DEVICES", help="CUDA_VISIBLE_DEVICES", type=int)
# parser.add_argument("WINDOW_SIZE", help="WINDOW_SIZE", type=int)
# parser.add_argument("N_FEATURES", help="N_FEATURES", type=int)
# args = parser.parse_args()
# print(args)

###########################################################
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# os.environ["CUDA_VISIBLE_DEVICES"]=str(args.CUDA_VISIBLE_DEVICES)

###########################################################
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))

###########################################################

import time
import glob
from collections import defaultdict
# from keras.layers import Dense, GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation, LSTMCell, GRUCell, Masking, TimeDistributed, BatchNormalization, Dropout, GaussianNoise
from keras.layers import Dense, Masking, TimeDistributed, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score
from keras.models import Model, Sequential, load_model
from tcn import TCN
from utils import load_train_history, save_train_history, plot_loss_history
from utils import evaluate_custom_metrics, CustomMetrics
from utils import voting_strategies, metrics_names, arch_to_str

# Set parameters
n_folds = 45
WINDOW_SIZE = 32
N_FEATURES = 6
# WINDOW_SIZE = args.WINDOW_SIZE
# N_FEATURES = args.N_FEATURES

dataset_type = f'{WINDOW_SIZE}ws_{N_FEATURES}f'
dataset_path_prefix = f'/home/ICT2000/jondras/datasets/vra1/subject_independent/{dataset_type}/'

folds_hist = []
for k in range(n_folds):

    # Load data for this fold
    data = np.load(dataset_path_prefix + f'{k}fold_{dataset_type}.npz')
    MASK_VALUE = data['MASK_VALUE']   

    X_train, Y_train = data['X_train'], data['Y_train']
    X_val,   Y_val   = data['X_val'],   data['Y_val']
    # X_test,  Y_test  = data['X_test'],  data['Y_test']

    #######################################################################################################
    # Create model
    
    n_epochs = 100
    batch_size = 128
    
    i = Input(batch_shape=(batch_size, WINDOW_SIZE, N_FEATURES))
    # Masking not supported!?
#     o = Masking(mask_value=MASK_VALUE)(i)
    o = TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], 
            use_skip_connections=True, dropout_rate=0.1, return_sequences=True, activation='linear', 
            use_batch_norm=False
           )(i)
    o = TimeDistributed(Dense(1, activation='sigmoid'))(o)

    model = Model(inputs=[i], outputs=[o])
    model.compile(loss='binary_crossentropy', optimizer='adam')
    if k == 0: 
        print(model.summary())

    #######################################################################################################
    # Train
    
    # Set early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

    # Checkpoint model weights and the model itself: at each epoch
#     model_type = f'{k}fold_{arch_to_str(GRU_ARCH)}u_{dataset_type}'
    model_type = f'{k}fold_TCN_{dataset_type}'
    print(f'INFO:\n\t{model_type}\n\t batch_size={batch_size}\n\t n_epochs={n_epochs}')
    model_checkpoint_path_prefix = f'./checkpoints/{model_type}/'
#     if not os.path.exists(model_checkpoint_path_prefix):
#         os.makedirs(model_checkpoint_path_prefix)
#     model_checkpoint_name = 'm_{epoch:04d}_{loss:.4f}_{val_loss:.4f}.hdf5'
#     model_checkpoint = ModelCheckpoint(model_checkpoint_path_prefix + model_checkpoint_name, monitor='val_loss', verbose=1, 
#                                        save_best_only=True, save_weights_only=False, mode='auto', period=1)

    # Pass validation chunks' lengths, for this fold
    custom_metrics = CustomMetrics(window_size=WINDOW_SIZE, val_lens=data['val_len'])
    hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=batch_size, epochs=n_epochs, verbose=1, 
                     callbacks=[
                         custom_metrics, 
#                          model_checkpoint, 
                         early_stop
                     ], shuffle=True)

    # Plot and save training history
    plot_loss_history(hist)
#     save_train_history(hist, model_type)

    #######################################################################################################
    # Save results from this fold
    curr_fold_hist = dict()
    curr_fold_hist.update(hist.history)
    curr_fold_hist.update(custom_metrics.metrics)
    folds_hist.append(curr_fold_hist)

#     break

# Save training history from all folds
# save_train_history(folds_hist, f'ALL_{n_folds}fold_{arch_to_str(GRU_ARCH)}u_{dataset_type}')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (128, 32, 6)         0                                            
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (128, 32, 64)        448         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (128, 32, 64)        8256        conv1d_21[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (128, 32, 64)        0           conv1d_22[0][0]                  
__________________________________________________________________________________________________
spatial_dr

Train on 159911 samples, validate on 28192 samples
Epoch 1/100
159872/159911 [============================>.] - ETA: 0s - loss: 0.2352

InvalidArgumentError: Incompatible shapes: [128] vs. [39]
	 [[{{node training/Adam/gradients/loss/time_distributed_2_loss/mul_grad/BroadcastGradientArgs}}]]
	 [[{{node loss/mul}}]]